# Tech Question: Lifetime

### **3. Identify “killer use case” for my technology (RRAM), Identify what the key limitations are of RRAM.**

What this question is asking for is to present one downside to RRAM: lifetime expectency. In our other research notebooks, we have presented multiple cases where RRAM has proven to be faster and more efficient than SRAM. However, we have not yet demonstrated how RRAM fails compared to SSRAM. 

SRAM is volatile memory technology, and therefore it does not store data once power is turned off. Thus, Lifetime and Reliability are not concerns with SRAM. However, RRAM is non-volatile memory technology which over time can erode and begin to fail, specifically when it is frequently used for applications with write heavy traffic patterns. 

In order to quantify exactly how poor the lifetime of RRAM is, we utlized the data from the 2016-2020_EnduranceSummary.csv. This csv has it has the minimum and maximum endurance for known NVMs like RRAM, a range which was determined based on a survey of many research papers. Even just looking at the csv, we can identify that RRAM has the lowest minimum and maximum endurance. 

As we do in all notebooks, we are import our libraries and set up the data we use for our research question.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, widgets, HBox, Layout
import ipywidgets as widgets
from IPython.display import display, Markdown
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CDSView, BooleanFilter
from bokeh.layouts import row

# Import Endurance Data 
endur_df = pd.read_csv('CSVFiles/2016-2020_EnduranceSummary.csv')

# These data types don't have min AND max data, so we're just dropping them because they're not a super useful comparison 
# If an evaluation of SOT and eFlash is needed, it would beneficial to create a graph that only compares the best endurance
cells_to_drop = ['SOT', 'eFlash', 'CTT'] # CTT has no data in the csv so for future use, we should implement that data
endur_df_filtered = endur_df[~endur_df['Memory Cell'].isin(cells_to_drop)]

The next block of code will create a graph that will visualize the max and min endurance of each NVM with user interaction involving Checkboxes.

In [9]:
#So the options the user can select are the unique memory cell types left after we drop SOT and eFlash 
tech_options = endur_df_filtered['Memory Cell'].unique()

#Create checkboxes for each of the possible memory cell types 
checkboxes = {tech: widgets.Checkbox(value=True, description=tech) for tech in tech_options}

#Function that takes in if the graph should be interactive and which checkboxes are checked 
def graph(TURN_ON, **kwargs):
    if TURN_ON:
        
        #Get the selected memory cell types from the kwargs
        selected_tech_targets = [tech for tech, selected in kwargs.items() if selected]
        plt.figure(figsize=(10, 6))
        
        #Graph selected memory technologies 
        for target in selected_tech_targets:
            specific_cell_data = endur_df_filtered[endur_df_filtered['Memory Cell'] == target]
            min_endurance = specific_cell_data['min']
            max_endurance = specific_cell_data['max']
            plt.hlines(target, min_endurance, max_endurance, color='skyblue', linewidth=10)
            plt.plot(min_endurance, target, "o", label="Min Endurance", color="blue")
            plt.plot(max_endurance, target, "o", label="Max Endurance", color="red")

        plt.xscale('log')
        plt.xlabel("Endurance (log scale)")
        plt.title("Endurance Range of Memory Cells")
        plt.legend(["Range of Endurance", "Min Endurance", "Max Endurance"])
        plt.show()
    
#Call graph with interactive on, checkboxes selected 
interact(graph, TURN_ON=True, **checkboxes)

interactive(children=(Checkbox(value=True, description='TURN_ON'), Checkbox(value=True, description='memristor…

<function __main__.graph(TURN_ON, **kwargs)>

To help visualize this downside, we created a graph that calculates the number of iterations that RRAM can withstand based on the max amount of write requests it takes before failing. Thus we need to import our RRAM Data.

In [10]:
# Intializing our RRAM Data in the same way as SRAM
RRAMdf = pd.read_csv('CSVFiles/RRAM_1BPC-combined.csv')
RRAMdf = RRAMdf = RRAMdf[RRAMdf["Area (mm^2)"] > 0]

# Concats both data frames together
dfs = pd.concat([RRAMdf])

# Filtering out negative Total Read Latency Data Since that is where most of our focus is on in this exploration
dfs = dfs[dfs['Total Read Latency (ms)'] >= 0]
dfs['OptimizationTarget'] = dfs['OptimizationTarget'].str.replace(' ', '') # There are some weird spaces in the Strings in this column so we filter them out


In [11]:
# Define the constant values for min_write_RR and max_write_RR- these are the number of cycles RRAM can do before it starts to be unreliable,
#we have quite the range because these estimations were generated using existing published work about lifespan of RRAM, gives us a best and worst case

#lifetime prediction
# Here is another case of specficed code, as we used data from published works to come up with the best and worst
# Number of writes for RRAM. For other NVMs, we would need to come up with their worst and best number of writes
#lifetime prediction, RRAM
min_write_RR = 10000       # Best case (min write RR)
max_write_RR = 1000000000 # Worst case (max write RR) - 1 billion


#Just for 16 MB 
MB16_benchmarks = dfs[dfs['Benchmark Name'].str.contains('16MB', case=False, na=False)]
unique_16 = MB16_benchmarks[['Benchmark Name', 'Write Accesses', 'Read Accesses']].drop_duplicates(subset='Benchmark Name').reset_index(drop = True)

# Finding the unique worst case benchmarks for 16MB
unique_16['max_iterations_worst_case'] = unique_16['Write Accesses'].apply(
    lambda x: min_write_RR / x if x != 0 else 0  # Use NaN for zero write accesses
)

#Find the unique best case benchmarks for 16MB
unique_16['max_iterations_best_case'] = unique_16['Write Accesses'].apply(
    lambda x: max_write_RR / x if x != 0 else 0  # Use NaN for zero write accesses
)


nan_count = unique_16['max_iterations_worst_case'].isna().sum()
nan_count_best = unique_16['max_iterations_best_case'].isna().sum()


  #get ratio of reads: writes for each benchmark
for index, r in unique_16.iterrows():
    # Access each row's values
    if r['Write Accesses'] != 0:
        unique_16.at[index, 'Reads/Write Accesses'] = r['Read Accesses'] / r['Write Accesses']
    else:
        unique_16.at[index, 'Reads/Write Accesses'] = r['Read Accesses']

nan_count_ratio = unique_16['Reads/Write Accesses'].isna().sum()

unique_16['min_iterations'] = unique_16['max_iterations_worst_case'].min
unique_16['max_iterations'] = unique_16['max_iterations_best_case'].max

#now, for 8MB 
MB8_benchmarks = dfs[dfs['Benchmark Name'].str.contains('8MB', case=False, na=False)]
unique_8 = MB8_benchmarks[['Benchmark Name', 'Write Accesses', 'Read Accesses']].drop_duplicates(subset='Benchmark Name').reset_index(drop = True)

unique_8['max_iterations_worst_case'] = unique_8['Write Accesses'].apply(
    lambda x: min_write_RR / x if x != 0 else 0  # Use NaN for zero write accesses
)

unique_8['max_iterations_best_case'] = unique_8['Write Accesses'].apply(
    lambda x: max_write_RR / x if x != 0 else 0  # Use NaN for zero write accesses
)


nan_count = unique_8['max_iterations_worst_case'].isna().sum()
nan_count_best = unique_8['max_iterations_best_case'].isna().sum()


  #get ratio of reads: writes for each benchmark
for index, r in unique_8.iterrows():
    # Access each row's values
    if r['Write Accesses'] != 0:
        unique_8.at[index, 'Reads/Write Accesses'] = r['Read Accesses'] / r['Write Accesses']
    else:
        unique_8.at[index, 'Reads/Write Accesses'] = r['Read Accesses']

nan_count_ratio = unique_8['Reads/Write Accesses'].isna().sum()

unique_8['min_iterations'] = unique_8['max_iterations_worst_case'].min
unique_8['max_iterations'] = unique_8['max_iterations_best_case'].max



# Create a checkbox for each Optimization Target
data_map = {
    '16MB Benchmarks': unique_16,
    '8MB Benchmarks': unique_8
}

checkboxes = {
    '16MB Benchmarks': widgets.Checkbox(value=True, description='16MB'),
    '8MB Benchmarks': widgets.Checkbox(value=True, description='8MB')
}

def graph(TURN_ON, **kwargs):
    if TURN_ON:
        # Get the selected memory cell types from the kwargs (checkbox states)
        selected_capacity = [capacity for capacity, selected in kwargs.items() if selected]
        plt.figure(figsize=(10, 6))

        for dataset_name in selected_capacity:
            unique_benchmarks = data_map[dataset_name]

            unique_benchmarks['max_iterations'] = pd.to_numeric(unique_benchmarks['max_iterations'], errors='coerce')
            unique_benchmarks['min_iterations'] = pd.to_numeric(unique_benchmarks['min_iterations'], errors='coerce')

            # Top 5 benchmarks with highest max iterations
            top_5_max = unique_benchmarks.nlargest(5, 'max_iterations')

            # Top 5 benchmarks with lowest min iterations
            top_5_min = unique_benchmarks.nsmallest(5, 'min_iterations')

            # Step 3: Combine the top 5 highest max iterations and top 5 lowest min iterations into one DataFrame
            top_benchmarks = pd.concat([top_5_max, top_5_min])

            # Step 4: Reset index for cleanliness
            top_benchmarks = top_benchmarks.reset_index(drop=True)

            # Plotting

            # Create a figure with more aesthetic settings
            fig, ax = plt.subplots(figsize=(12, 7))

            # Get the positions for the bars on the y-axis
            y_pos = np.arange(len(top_benchmarks))

            # Set bar height
            bar_height = 0.35

            # Prettier pastel colors for the bars
            best_case_color = '#A3E4D7'  # Soft mint green
            worst_case_color = '#F5B7B1'  # Soft blush pink

            # Plot max iterations for best-case and worst-case (horizontal bars now)
            ax.barh(y_pos - bar_height/2, top_benchmarks['max_iterations_best_case'], bar_height, label='Best Case Max Iterations', color=best_case_color, edgecolor='black', linewidth=0.8, zorder=2)
            ax.barh(y_pos + bar_height/2, top_benchmarks['max_iterations_worst_case'], bar_height, label='Worst Case Max Iterations', color=worst_case_color, edgecolor='black', linewidth=0.8, zorder=2)

            # Customize the plot
            ax.set_yticks(y_pos)
            ax.set_yticklabels(top_benchmarks['Benchmark Name'], fontsize=12, fontweight='bold')  # Benchmark names on y-axis
            ax.set_xlabel('Max Iterations (Log Scale)', fontsize=14, fontweight='bold')
            ax.set_ylabel('Benchmark', fontsize=14, fontweight='bold')
            ax.set_title('Top 5 Benchmarks with Highest Max Iterations & Lowest Min Iterations', fontsize=16, fontweight='bold')

            # Set x-axis to log scale
            ax.set_xscale('log')

            #add a line at 1 iteration
            ax.axvline(x=1, color='black', linestyle='--', linewidth=1, label='x = 1 line')

            # Add a legend outside the plot area, to the right
            ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=12, title="Scenario", title_fontsize=13, shadow=True)

            # Style the grid for a cleaner look
            ax.grid(True, axis='x', linestyle='--', alpha=0.6, zorder=1)

            # Step 5: Adjust layout to prevent overlap
            plt.tight_layout()

            # Display the plot
            plt.show()


output_notebook()
interact(graph, TURN_ON=True, **checkboxes)

Loading BokehJS ...

interactive(children=(Checkbox(value=True, description='TURN_ON'), Checkbox(value=True, description='16MB'), C…

<function __main__.graph(TURN_ON, **kwargs)>

To get a better sense of how RRAM's poor lifetime would impact the functionality of different applications, based on memory access patterns, we calculated the number of iterations you could do of each benchmark using one RRAM cell, under worst case lifetime estimates and best-case lifetime estimates. We then found the 10 benchmarks with the 5 worst worse case max iterations (top five above) and the 5 best best case max iterations (bottom five) to give a sense of the limitations of RRAM. The vertical line shows where the cutoff is for a single iteration of the benchmark on the y axis. As you can see, in the best case scenario, RRAM can sustain upwards of 10^5 iterations of certain benchmarks. However, under pessemistic assumptions, there are some traffic patterns that it can complete less than a single iteration before failing. Some potential use cases based on the benchmarks with low read amounts are combniatorial optimization, and ray tracing.

In [12]:
df4 = dfs.copy(deep='True')
df4 = df4[df4['Benchmark Name'] != 'test']
#544.nab_r64MB is mislabelled in the dataset, it should be test instead, dropping duplicates gets rid of it though
df4['Visible'] = True
def findCapacity(x):
    if '8MB' in x:
        return '8'
    elif '16MB' in x:
        return '16'
    elif '32MB' in x:
        return '32'
    elif '64MB' in x:
        return '64'
    else:
        return ''
df4['Capacity2'] = df4['Benchmark Name'].astype(str).apply(findCapacity)
df4 = df4.drop_duplicates(subset=['Benchmark Name', 'Capacity2'])
capacities = sorted(df4['Capacity2'].unique().astype(str).tolist(), key=int)
df4['ratio'] = np.log10(df4['Write Accesses']/df4['Read Accesses'])
df4['time'] = df4['Total Read Latency (ms)'] + df4['Total Write Latency (ms)']
df4 = df4[df4['Write Accesses'] != 0] 
best = df4.copy(deep='True')
worst = df4.copy(deep='True')
best['cycles'] = 1000000000/df4['Write Accesses']
worst['cycles'] = 10000/df4['Write Accesses']
best['time'] = best['time']*best['cycles']
worst['time'] = worst['time']*worst['cycles']
best['Write Accesses'] = np.log10(best['Write Accesses'])
worst['Write Accesses'] = np.log10(worst['Write Accesses'])
best = ColumnDataSource(best)
worst = ColumnDataSource(worst)
figures = []
TOOLTIPS = [('Benchmark', '@{Benchmark Name}'), ('Iterations', '@{cycles}')]
hvr = HoverTool(tooltips=TOOLTIPS)

def graph(**kwargs):
    figures = []
    selected_opt_targets = [opt_t for opt_t, selected in kwargs.items() if selected]
    for target in selected_opt_targets:
        bestVisible = [True if visibility == True else False for visibility in best.data['Visible']]
        bestTarget = [True if str(capacity) == target else False for capacity in best.data['Capacity2']]
        worstVisible = [True if visibility == True else False for visibility in worst.data['Visible']]
        worstTarget = [True if capacity == target else False for capacity in worst.data['Capacity2']]
        bestView = CDSView(filter=BooleanFilter(bestVisible) & BooleanFilter(bestTarget))
        worstView = CDSView(filter=BooleanFilter(worstVisible) & BooleanFilter(worstTarget))
        fig = figure(title = target + "MB", width=300, height=300, background_fill_color="#fafafa",
               y_axis_type="log", x_axis_label = 'log(Write Accesses)')
        fig.scatter('Write Accesses', 'cycles', size=10, fill_color='#0e14b5', 
              legend_label='Best Case', source = best, view=bestView)
        fig.scatter('Write Accesses', 'cycles', size=10, fill_color='#d40820', 
              legend_label='Worst Case', source = worst, view=worstView)
        fig.add_tools(hvr)
        figures.append(fig)
    if len(figures) > 0:
        figures[0].yaxis.axis_label = "Possible Iterations"
        figures[0].yaxis.axis_label_text_font_size = "18pt"
    show(row(figures))
    
display(Markdown("# Possible Iterations based on Write Access for RRAM Benchmarks"))
checkboxes = {opt_t: widgets.ToggleButton(value=True, description=str(opt_t)+"MB") for opt_t in capacities}
panel = interactive(graph, **checkboxes)
display(HBox(panel.children[:-1], layout = Layout(flex_flow='row wrap')))
output_notebook()
display(panel.children[-1])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Possible Iterations based on Write Access for RRAM Benchmarks

Loading BokehJS ...

Output()

# Conclusions #

#### RRAM is primarily designed for read operations and performs well in this regard. Its main limitation arises when used for write operations, which can be less reliable. Despite this, there are configurations that can extend the lifespan of RRAM cells, especially under conditions of minimal write activity. If optimistic assumptions about cell lifetime hold true, RRAM can be a viable option even for write-heavy tasks.